# https://github.com/dreamquark-ai/tabnet

In [1]:
from google.colab import drive
drive.mount('./content/')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!pip install  "git+https://github.com/dreamquark-ai/tabnet.git@develop#egg=pytorch_tabnet" --upgrade
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization

# Label Encoding

## 전처리

In [2]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
import random
import os
from sklearn.metrics import log_loss
# from bayes_opt import BayesianOptimization
# import lightgbm
import torch
from pytorch_tabnet.tab_model import TabNetClassifier

d = "C:\kaggle_data\credit_card"
lst = os.listdir(d)
print(lst)
train = pd.read_csv(d + '\\' +lst[3])
test = pd.read_csv(d + '\\' +lst[2])
ss = pd.read_csv(d + '\\' +lst[1])
train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 
test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)

# Married, Civil marriage
train['income_per_size'] = np.log(train['income_total']/train['family_size'])
test['income_per_size'] = np.log(test['income_total']/test['family_size'])
train.loc[(train['family_type']=='Married')|(train['family_type']=='Civil marriage'),'income_per_size']\
= train['income_per_size'] * 2

test.loc[(test['family_type']=='Married')|(test['family_type']=='Civil marriage'),'income_per_size']\
= test['income_per_size'] * 2

def simple_marry(x):
    if x == 'Married' or x =='Civil marriage':
        return '0'
    elif x == 'Separated' or x == 'Widow':
        return '1'
    else:
        return '2'

for df in [train,test]:
    df['family_bins'] = df['family_type'].apply(simple_marry)

# income_total을 로그변환
train['income_total'] = np.log(train['income_total'])
# train = train.drop('income_total',1)
test['income_total'] = np.log(test['income_total'])
# test = test.drop('income_total',1)

# car와 reality를 합친 새로운 칼럼 careality
train['car'] =train['car'].apply(lambda x: int(x=='Y'))
train['reality'] =train['reality'].apply(lambda x: int(x=='Y'))
test['car'] =test['car'].apply(lambda x: int(x=='Y'))
test['reality'] =test['reality'].apply(lambda x: int(x=='Y'))

train['careality'] = train['car'] + train['reality']
train = train.drop(['car', 'reality'],1)

test['careality'] = test['car'] + test['reality']
test = test.drop(['car', 'reality'],1)

train.drop('gender',1,inplace=True)
test.drop('gender',1,inplace=True)

object_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)
###############################
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train.drop(object_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)

test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test.drop(object_col, axis=1, inplace=True)
test = pd.concat([test, test_onehot_df], axis=1)
########################################




## 제곱
for df in [train,test]:
    df['income_per_size'] = df['income_per_size'].apply(lambda x: x**2)
    
c = 'income_per_size'
mean = train[c].mean()
std = train[c].std()

# 기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std

#####################
c = 'income_total'
k = 2.2
mean = train[c].mean()
std = train[c].std()
idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
train = train.drop(idxs).reset_index(drop=True)

# 기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std

#####################
c = 'begin_month'
k = 2.2

train[c] = train[c].apply(lambda x: x**2)
test[c] = test[c].apply(lambda x: x**2)

mean = train[c].mean()
std = train[c].std()

idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
# train = train.drop(idxs).reset_index(drop=True)

# 기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std

######################
c = 'DAYS_BIRTH'
k = 2.2
mean = train[c].mean()
std = train[c].std()
idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
train = train.drop(idxs).reset_index(drop=True)

#기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std



###################
c = 'DAYS_EMPLOYED'
k = 2.2
mean = train[c].mean()
std = train[c].std()
idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
train = train.drop(idxs).reset_index(drop=True)



ran = ['child_num', 'DAYS_BIRTH', 'DAYS_EMPLOYED','family_size', 'begin_month', 'income_per_size']
from sklearn.preprocessing import QuantileTransformer
tf = QuantileTransformer(n_quantiles=100,random_state=42, output_distribution='normal')
tf.fit(train[ran])
train[ran] = tf.transform(train[ran])
test[ran] = tf.transform(test[ran])



MAX_EPOCHS = 1000
BATCH_SIZE = 1024
VIRTUAL_BATCH_SIZE = 128
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Using {}".format(DEVICE))

['.ipynb_checkpoints', 'sample_submission.csv', 'test.csv', 'train.csv', 'Untitled.ipynb']
Using cuda


In [3]:
# cat_idxs = [ i for i, f in enumerate(train.columns) if f in object_col]

# cat_dims = [len(total.iloc[:,idx].unique()) for idx in cat_idxs]

In [4]:
n = 10
skf = StratifiedKFold(n_splits=n, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))
random.seed(42)

- n_step: 4,6

## 파라미터 튜닝 완료

In [ ]:
# models={}
# for fold in range(n):
#     train_idx, valid_idx = folds[fold]
#     X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
#                                       train['credit'][train_idx].values, train['credit'][valid_idx].values 

#     clf = TabNetClassifier(n_d=9, n_a=9,
#                                     n_steps=6,
#                                     gamma=1.006,
#                                     n_independent=4,
#                                     n_shared=4,
#                                     lambda_sparse=0.01994,
#                                     seed=42,

#                                     optimizer_fn=torch.optim.Adam,
#                                     optimizer_params=dict(lr=1e-2), 
#                                     scheduler_params = {"gamma": 0.95,
#                                       "step_size": 20},
#                                     scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15,
#                                     device_name = 'auto',
#                                     verbose=0)
#     clf.fit(
#                   X_train=X_train, y_train=y_train,
#                   eval_set=[(X_valid, y_valid)],
#                   eval_metric=['logloss'],
#                   max_epochs=MAX_EPOCHS ,
#                   patience=50, # please be patient ^^
#                   batch_size=32768,
#                   virtual_batch_size=16384,
#                   num_workers=0)
#     models[fold] = clf

In [ ]:
models={}
for fold in range(n):
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                      train['credit'][train_idx].values, train['credit'][valid_idx].values 

    clf = TabNetClassifier(n_d=9, n_a=9,
                                    n_steps=6,
                                    gamma=1.006,
                                    n_independent=4,
                                    n_shared=4,
                                    lambda_sparse=0.01994,
                                    seed=42,

                                    optimizer_fn=torch.optim.Adam,
                                    optimizer_params=dict(lr=1e-2), 
                                    scheduler_params = {"gamma": 0.95,
                                      "step_size": 20},
                                    scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15,
                                    device_name = 'auto',
                                    verbose=0)
    clf.fit(
                  X_train=X_train, y_train=y_train,
                  eval_set=[(X_valid, y_valid)],
                  eval_metric=['logloss'],
                  max_epochs=MAX_EPOCHS ,
                  patience=50, # please be patient ^^
                  batch_size=32768,
                  virtual_batch_size=16384,
                  num_workers=0)
    models[fold] = clf

In [ ]:
models={}
for fold in range(n):
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                      train['credit'][train_idx].values, train['credit'][valid_idx].values 

    clf = TabNetClassifier(device_name = DEVICE,verbose=0)
    clf.fit(
                  X_train=X_train, y_train=y_train,
                  eval_set=[(X_valid, y_valid)],
                  eval_metric=['logloss'],
                  max_epochs=MAX_EPOCHS ,
                  patience=50, # please be patient ^^
                  batch_size=32768,
                  virtual_batch_size=16384,
                  num_workers=0)
    models[fold] = clf

In [ ]:
ss.iloc[:,1:]=0
for fold in range(n):
    ss.iloc[:,1:] += models[fold].predict_proba(test.values)/n

In [ ]:
ss.to_csv('05_22_tabnet.csv',index=False)

In [ ]:
from tqdm import tqdm
# loss=0
# models={}
# historys={}
results = []
for n_step in [5,6,9]:
    for ind in range(1,6):
        for sha in range(1,6):
            loss=0
            models={}
            historys={}
            for fold in tqdm(range(n)):
                train_idx, valid_idx = folds[fold]
                X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                                  train['credit'][train_idx].values, train['credit'][valid_idx].values 
                clf = TabNetClassifier(n_d=9, n_a=9,
                                n_steps=6
                                gamma=1.006,
                                n_independent=4,
                                n_shared=4,
                                lambda_sparse=0.01994,
                                seed=42,

                                optimizer_fn=torch.optim.Adam,
                                optimizer_params=dict(lr=1e-2), 
                                scheduler_params = {"gamma": 0.95,
                                  "step_size": 20},
                                scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15,
                                device_name = 'auto',
                                verbose=0)

                history = clf.fit(
                  X_train=X_train, y_train=y_train,
                  eval_set=[(X_valid, y_valid)],
                  eval_metric=['logloss'],
                  max_epochs=MAX_EPOCHS ,
                  patience=50, # please be patient ^^
                  batch_size=32768,
                  virtual_batch_size=16384,
                  num_workers=0)
                models[fold] = clf
                historys[fold]= history
                # y_pred = clf.predict_proba(X_valid.values)
                y_pred = clf.predict_proba(X_valid)
                loss += log_loss(y_valid,y_pred)
                results.append([n_step,ind,sha])
                print(n_step,ind,sha)

In [ ]:
train.columns

In [ ]:
loss/10

In [ ]:
models[0]

## drdrdr 모델링, 탐색

In [ ]:
##### 파라미터 최적화 탐색 중
# n_d, n_a == 9, 16
# lambda_sparse == 0.0001, 0.1478, 0.01994 (9일 떄)
# decision_step == 
##### 
n = 4
skf = StratifiedKFold(n_splits=n, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))
random.seed(42)

max_epochs = 1000
models={}
def eval_tabnet(n_independent):
  loss=0
  for fold in range(n):
      train_idx, valid_idx = folds[fold]
      X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                          train['credit'][train_idx].values, train['credit'][valid_idx].values 
      
      clf = TabNetClassifier(n_d=9, n_a=9,
                        n_steps=5,
                        gamma=1.006,
#                         n_independent=int(n_independent),
                        #  n_shared=int(n_shared),
                        # momentum=0.3, 
                        # clip_value=2., 
                             
                        lambda_sparse=0.01994,
                        seed=42,
                        optimizer_fn=torch.optim.Adam,
                        optimizer_params=dict(lr=1e-2), 
                        scheduler_params = {"gamma": 0.95,
                          "step_size": 20},
                        scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15,
                        device_name = 'auto',
                        verbose=0)
      
      clf.fit(
          X_train=X_train, y_train=y_train,
          eval_set=[(X_valid, y_valid)],
      #     eval_name=None,
          eval_metric=['logloss'],
      #     loss_fn=None,
          max_epochs=max_epochs ,
          patience=50, # please be patient ^^
          batch_size=1024,
          virtual_batch_size=256,
          num_workers=1,
          )
      models[fold] = clf
      # y_pred = clf.predict_proba(X_valid.values)
      y_pred = clf.predict_proba(X_valid)
      loss += log_loss(y_valid,y_pred)
  return -loss/n

n_d9일떄 gamma찾는 중

In [ ]:
lgbBO = BayesianOptimization(eval_tabnet, {'n_independent': (1,5),
                                        # 'n_d': (9,16)
                                        # 'min_data_in_leaf': (100, 2000),
                                        
                                          })
lgbBO.maximize(n_iter=20, init_points=2)

In [ ]:
models={}
for fold in range(n):
  print(n+1,'번째')
  train_idx, valid_idx = folds[fold]
  X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                      train['credit'][train_idx].values, train['credit'][valid_idx].values 
  
  clf = TabNetClassifier(n_d=int(9), n_a=int(9),
                    # n_steps=3,
                    # gamma=1.0,# n_independent=2, n_shared=2,
                    # lambda_sparse=1e-4,
                    cat_idxs=cat_idxs,
                      cat_dims=cat_dims,
                      cat_emb_dim=1,
                    seed=42,
                    # momentum=0.3, 
                    # clip_value=2., 
                    optimizer_fn=torch.optim.Adam,
                    optimizer_params=dict(lr=1e-2), 
                    scheduler_params = {"gamma": 0.95,
                      "step_size": 20},
                    scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15,
                    device_name = 'auto',
                    verbose=0)
  clf.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_valid, y_valid)],
    #     eval_name=None,
        eval_metric=['logloss'],
    #     loss_fn=None,
        max_epochs=max_epochs ,
        patience=50, # please be patient ^^
        batch_size=1024,
        virtual_batch_size=256,
        num_workers=1)
  models[fold] = clf

In [ ]:
clf = TabNetClassifier(n_d=int(9), n_a=int(9),
                  # n_steps=3,
                  # gamma=1.0,# n_independent=2, n_shared=2,
                  # lambda_sparse=1e-4,
                  cat_idxs=cat_idxs,
                    cat_dims=cat_dims,
                    cat_emb_dim=1,
                  seed=42,
                  # momentum=0.3, 
                  # clip_value=2., 
                  optimizer_fn=torch.optim.Adam,
                  optimizer_params=dict(lr=1e-2), 
                  scheduler_params = {"gamma": 0.95,
                    "step_size": 20},
                  scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15,
                  device_name = 'auto',
                  verbose=0)
clf.fit(
      X_train=X_train, y_train=y_train,
      eval_set=[(X_valid, y_valid)],
  #     eval_name=None,
      eval_metric=['logloss'],
  #     loss_fn=None,
      max_epochs=max_epochs ,
      patience=50, # please be patient ^^
      batch_size=1024,
      virtual_batch_size=256,
      num_workers=1)

In [ ]:
preds = clf.predict_proba(test.values)
preds

In [ ]:
ss.iloc[:,1:]=0
ss.iloc[:,1:] += preds
ss

In [ ]:
ss.iloc[:,1:]=0
for fold in range(n):
    ss.iloc[:,1:] += models[fold].predict_proba(test.values)/n

In [ ]:
ss.to_csv('tabnet_test.csv', index=False) # 0.7272812144